In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import gc

In [ ]:
url_source = "https://www.rocketpunch.com/companies?page="

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [ ]:
driver = webdriver.Chrome("C:/chromedriver.exe", chrome_options=options)
driver.get("https://www.rocketpunch.com/companies?page="+str(1)+"&q=")
df_list = list()

for n in range(2,3398):
    time.sleep(2.5)
    gc.collect()
    html = driver.page_source
    bs_xml = bs(html, "html.parser")
    a = bs_xml.find("div", {"id":"company-list"}).findAll("div", {"class":"company item"})
    for _ in a:
        cp_name = _.find('img')['alt'].split(" ")[0] 
        cp_url = 'https://www.rocketpunch.com' + _.find("a",{'class':'link'})['href']
        df_list.append([cp_name, cp_url])
    print(n, " 완료")
    try:
        driver.find_element_by_css_selector('#search-results > div.ui.blank.right.floated.segment > div > a:nth-child(4) > i').click()
    except:
        driver.find_element_by_xpath('//*[@id="company_signup_popup"]/div[1]/i').click()
        print("팝업처리")
        time.sleep(0.5)
        driver.find_element_by_css_selector('#search-results > div.ui.blank.right.floated.segment > div > a:nth-child(4) > i').click()
        
df = pd.DataFrame(df_list, columns = ['name', 'url'])

In [ ]:
df = pd.DataFrame(df_list, columns = ['name', 'url'])

In [ ]:
df.to_csv("RocketPunch.csv", index = False)

In [ ]:
driver.get_screenshot_as_file('error.png')

In [ ]:
df_org = pd.DataFrame(df.groupby('name')['url'].apply(list))

In [ ]:
df_org.reset_index(inplace = True)

In [ ]:
df_org['url'] = df_org.url.apply(lambda x : x[0])

In [ ]:
df_org